<a href="https://colab.research.google.com/github/dibsondivya/ai-health-event/blob/main/AI_Models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Source

Code Outline taken from: [Google Colab](https://colab.research.google.com/github/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb#scrollTo=lnwgmqNG7i5l
)
 


**Range of models attempted**

Successful models include:
* check

Failed models due to packaging error include:
* [Autotrain Mental Health Analysis](https://huggingface.co/rabiaqayyum/autotrain-mental-health-analysis-752423172)
* [Healthsea](https://huggingface.co/explosion/en_healthsea)

# Package Installation

In [ ]:
!pip install sentencepiece==0.1.94
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 35.9 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 31.8 MB/s 
     |████████████████████████████████| 86 kB 5.5 MB/s 
     |████████████████████████████████| 6.6 MB 59.0 MB/s 
     |████████████████████████████████| 596 kB 63.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss
import torch
import transformers as ppb
import warnings
warnings.filterwarnings('ignore')

# Dataset Import

In [ ]:
df = pd.read_csv("train_textcleaned.csv")
df

,Unnamed: 0,tweet,label
0,1,New Alzheimer Society resources to support a m...,1
1,4,Hippocampus ! If this part are damage we could...,1
2,5,How tackling hearing loss could reduce your ri...,1
3,6,Finding the fun in the fog of Alzheimer’s via ...,1
4,8,A family of 6 drowned in a van. 2 adults (with...,1
...,...,...,...
3170,4532,"Eyes on her feet, lips on her toes. Now keep q...",0
3171,4533,When I watch Trump's vids w/o sound...I feel l...,0
3172,4534,Bruh i almost had a heat stroke coming to work,1
3173,4535,after i stroke her leave her pussy soakin😋,0


In [ ]:
df['label'].value_counts()

1    1719
0     876
2     366
3     214
Name: label, dtype: int64

In [ ]:
df_test = pd.read_csv("test_textcleaned.csv")
df_test

,Unnamed: 0,tweet,label
0,2,What to Know About Thyroid and Alzheimer's,1
1,3,"一瞬「？」でしたが、なるほどプリオンの絡みか。 Alzheimer, Parkinson D...",1
2,7,Encouraging!!! Study reveals what causes Alzhe...,1
3,10,Taking statins for 6 months reduces risk of Al...,1
4,11,The Telegraph will give £1 to Alzheimer's Rese...,1
...,...,...,...
1356,4518,Timing of Carotid Revascularization Procedures...,1
1357,4520,MXA BUILDS A FULL-RACE 2006 CR250 TWO-STROKE -...,0
1358,4523,"Psst, at the stroke of midnight, go wish @Sman...",0
1359,4528,"@NoceurTitanium ""Depends on the lighting."" She...",0


In [ ]:
df_test['label'].value_counts()

1    737
0    392
2    146
3     86
Name: label, dtype: int64

# DistilBERT

In [ ]:
# For DistilBERT:
model_class, tokenizer_class, pretrained_weights = (ppb.DistilBertModel, ppb.DistilBertTokenizer, 'distilbert-base-uncased')

## Want BERT instead of distilBERT? Uncomment the following line:
#model_class, tokenizer_class, pretrained_weights = (ppb.BertModel, ppb.BertTokenizer, 'bert-base-uncased')

# Load pretrained model/tokenizer
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized = df['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,0,:].numpy()
labels = df['label']

In [ ]:
tokenized_test = df_test['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)

padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])

np.array(padded_test).shape

attention_mask_test = np.where(padded_test != 0, 1, 0)
attention_mask_test.shape

input_ids_test = torch.tensor(padded_test)  
attention_mask_test = torch.tensor(attention_mask_test)

with torch.no_grad():
    last_hidden_states = model(input_ids_test, attention_mask=attention_mask_test)

features_test = last_hidden_states[0][:,0,:].numpy()
labels_test = df_test['label']

## Without parameter grid search

### Validation Set

In [ ]:
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.3, random_state=11)

### Get accuracy and loss on validation

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(val_features, val_labels)

0.8163693599160545

In [ ]:
log_loss(val_labels, lr_clf.predict_proba(val_features)) 
# second argument gives prob estimates for each class

0.5097167447841822

### Get accuracy and loss on test

In [ ]:
lr_clf.score(features_test, labels_test)

0.8354151359294636

In [ ]:
log_loss(labels_test, lr_clf.predict_proba(features_test)) 

0.479673244760273

In [ ]:
from google.colab import files
predictions = lr_clf.predict(features_test)
pd.DataFrame(predictions, columns=['predictions']).to_csv('DistilBERT_testprediction.csv')
files.download('DistilBERT_testprediction.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## With parameter grid search

In [ ]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

best parameters:  {'C': 100.0}
best scores:  0.7794736309343051


### Get accuracy and loss on validation

In [ ]:
lr_clf_grid = LogisticRegression(C=100)
lr_clf_grid.fit(train_features, train_labels) # Fit the model according to the given training data.
lr_clf_grid.score(val_features, val_labels) # get average accuracy

0.8006295907660022

In [ ]:
log_loss(val_labels, lr_clf_grid.predict_proba(val_features)) 
# second argument gives prob estimates for each class

0.7731661744285423

### Get accuracy and loss on test

In [ ]:
lr_clf_grid.score(features_test, labels_test)

0.813372520205731

In [ ]:
log_loss(labels_test, lr_clf_grid.predict_proba(features_test)) 
# second argument gives prob estimates for each class

0.7491628932198535

## Compare against dummy

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.537 (+/- 0.00)


# Ernie 2.0

In [ ]:
# Load pretrained model/tokenizer
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("nghuyong/ernie-2.0-en")
model = AutoModel.from_pretrained("nghuyong/ernie-2.0-en")

Downloading:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/479 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at nghuyong/ernie-2.0-en were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
tokenized = df['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,0,:].numpy()
labels = df['label']

In [ ]:
tokenized_test = df_test['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)

padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])

np.array(padded_test).shape

attention_mask_test = np.where(padded_test != 0, 1, 0)
attention_mask_test.shape

input_ids_test = torch.tensor(padded_test)  
attention_mask_test = torch.tensor(attention_mask_test)

with torch.no_grad():
    last_hidden_states = model(input_ids_test, attention_mask=attention_mask_test)

features_test = last_hidden_states[0][:,0,:].numpy()
labels_test = df_test['label']

## Without parameter grid search

### Validation Set

In [ ]:
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.3, random_state=11)

### Get accuracy and loss on validation set

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(val_features, val_labels)

0.7932843651626443

In [ ]:
log_loss(val_labels, lr_clf.predict_proba(val_features)) 
# second argument gives prob estimates for each class

0.6942063505562114

### Get accuracy and loss on test set

In [ ]:
lr_clf.score(features_test, labels_test)

0.8008817046289493

In [ ]:
log_loss(labels_test, lr_clf.predict_proba(features_test)) 
# second argument gives prob estimates for each class

0.627507252054266

## With parameter grid search

In [ ]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

best parameters:  {'C': 5.263252631578947}
best scores:  0.7812774572325134


### Get accuracy and loss on validation set

In [ ]:
lr_clf_grid = LogisticRegression(C=5.263252631578947)
lr_clf_grid.fit(train_features, train_labels)
lr_clf_grid.score(val_features, val_labels)

0.7649527806925498

In [ ]:
log_loss(val_labels, lr_clf_grid.predict_proba(val_features)) 
# second argument gives prob estimates for each class

1.489674821904077

### Get accuracy and loss on test set

In [ ]:
lr_clf_grid.score(features_test, labels_test)

0.7905951506245408

In [ ]:
log_loss(labels_test, lr_clf_grid.predict_proba(features_test)) 
# second argument gives prob estimates for each class

1.3505694356966296

## Compare against dummy

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.542 (+/- 0.00)


# BigBird Health Fact

In [ ]:
# Load pretrained model/tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model = AutoModelForSequenceClassification.from_pretrained("nbroad/bigbird-base-health-fact")

tokenizer = AutoTokenizer.from_pretrained("nbroad/bigbird-base-health-fact")

Downloading:   0%|          | 0.00/1.16k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/489M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/826k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.34M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/775 [00:00<?, ?B/s]

In [ ]:
tokenized = df['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,:].numpy()
labels = df['label']

Attention type 'block_sparse' is not possible if sequence_length: 56 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


In [ ]:
tokenized_test = df_test['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)

padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])

np.array(padded_test).shape

attention_mask_test = np.where(padded_test != 0, 1, 0)
attention_mask_test.shape

input_ids_test = torch.tensor(padded_test)  
attention_mask_test = torch.tensor(attention_mask_test)

with torch.no_grad():
    last_hidden_states = model(input_ids_test, attention_mask=attention_mask_test)

features_test = last_hidden_states[0][:,:].numpy()
labels_test = df_test['label']

## Without parameter grid search

### Validation Set

In [ ]:
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.3, random_state=11)

### Get accuracy and loss on validation set

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(val_features, val_labels)

0.5865687303252886

In [ ]:
log_loss(val_labels, lr_clf.predict_proba(val_features)) 
# second argument gives prob estimates for each class

1.0311218493027876

### Get accuracy and loss on test set

In [ ]:
lr_clf.score(features_test, labels_test)

0.6069066862601029

In [ ]:
log_loss(labels_test, lr_clf.predict_proba(features_test)) 
# second argument gives prob estimates for each class

0.9953715194411588

## With parameter grid search

In [ ]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

best parameters:  {'C': 73.68423684210526}
best scores:  0.5994624962040692


### Get accuracy and loss on validation set

In [ ]:
lr_clf_grid = LogisticRegression(C=73.68423684210526)
lr_clf_grid.fit(train_features, train_labels)
lr_clf_grid.score(val_features, val_labels)

0.5876180482686254

In [ ]:
log_loss(val_labels, lr_clf_grid.predict_proba(val_features)) 
# second argument gives prob estimates for each class

1.030220801733119

### Get accuracy and loss on test set

In [ ]:
lr_clf_grid.score(features_test, labels_test)

0.6061719324026451

In [ ]:
log_loss(labels_test, lr_clf_grid.predict_proba(features_test)) 
# second argument gives prob estimates for each class

0.997324668096514

## Compare against dummy

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.537 (+/- 0.00)


# Clinical Mortality Prediction

https://huggingface.co/bvanaken/CORe-clinical-mortality-prediction

In [ ]:
# Load pretrained model/tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("bvanaken/CORe-clinical-mortality-prediction")
model = AutoModelForSequenceClassification.from_pretrained("bvanaken/CORe-clinical-mortality-prediction")

Downloading:   0%|          | 0.00/844 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

In [ ]:
tokenized = df['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,:].numpy()
labels = df['label']

In [ ]:
tokenized_test = df_test['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)

padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])

np.array(padded_test).shape

attention_mask_test = np.where(padded_test != 0, 1, 0)
attention_mask_test.shape

input_ids_test = torch.tensor(padded_test)  
attention_mask_test = torch.tensor(attention_mask_test)

with torch.no_grad():
    last_hidden_states = model(input_ids_test, attention_mask=attention_mask_test)

features_test = last_hidden_states[0][:,:].numpy()
labels_test = df_test['label']

## Without parameter grid search

### Validation Set

In [ ]:
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.3, random_state=11)

### Get accuracy and loss on validation set

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(val_features, val_labels)

0.5487932843651626

In [ ]:
log_loss(val_labels, lr_clf.predict_proba(val_features)) 
# second argument gives prob estimates for each class

1.1192464872926762

### Get accuracy and loss on test set

In [ ]:
lr_clf.score(features_test, labels_test)

0.5415135929463629

In [ ]:
log_loss(labels_test, lr_clf.predict_proba(features_test)) 
# second argument gives prob estimates for each class

1.0834868391210033

## With parameter grid search

In [ ]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

best parameters:  {'C': 0.0001}
best scores:  0.5373539831966798


### Get accuracy and loss on validation set

In [ ]:
lr_clf_grid = LogisticRegression(C=0.0001)
lr_clf_grid.fit(train_features, train_labels)
lr_clf_grid.score(val_features, val_labels)

0.5508919202518363

In [ ]:
log_loss(val_labels, lr_clf_grid.predict_proba(val_features)) 
# second argument gives prob estimates for each class

1.1352413144081277

### Get accuracy and loss on test set

In [ ]:
lr_clf_grid.score(features_test, labels_test)

0.5415135929463629

In [ ]:
log_loss(labels_test, lr_clf_grid.predict_proba(features_test)) 
# second argument gives prob estimates for each class

1.1037502081137909

## Compare against dummy

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.537 (+/- 0.00)


# Clinical Assertion Negation BERT

https://huggingface.co/bvanaken/clinical-assertion-negation-bert

In [ ]:
# Load pretrained model/tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("bvanaken/clinical-assertion-negation-bert")
model = AutoModelForSequenceClassification.from_pretrained("bvanaken/clinical-assertion-negation-bert")

Downloading:   0%|          | 0.00/819 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/413M [00:00<?, ?B/s]

In [ ]:
tokenized = df['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,:].numpy()
labels = df['label']

In [ ]:
tokenized_test = df_test['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)

padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])

np.array(padded_test).shape

attention_mask_test = np.where(padded_test != 0, 1, 0)
attention_mask_test.shape

input_ids_test = torch.tensor(padded_test)  
attention_mask_test = torch.tensor(attention_mask_test)

with torch.no_grad():
    last_hidden_states = model(input_ids_test, attention_mask=attention_mask_test)

features_test = last_hidden_states[0][:,:].numpy()
labels_test = df_test['label']

## Without parameter grid search

### Validation Set

In [ ]:
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.3, random_state=11)

### Get accuracy and loss on validation set

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(val_features, val_labels)

0.5393494228751312

In [ ]:
log_loss(val_labels, lr_clf.predict_proba(val_features)) 
# second argument gives prob estimates for each class

1.115070899042313

### Get accuracy and loss on test set

In [ ]:
lr_clf.score(features_test, labels_test)

0.5312270389419544

In [ ]:
log_loss(labels_test, lr_clf.predict_proba(features_test)) 
# second argument gives prob estimates for each class

1.0781601562752412

## With parameter grid search

In [ ]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

best parameters:  {'C': 73.68423684210526}
best scores:  0.5463579309646726


### Get accuracy and loss on validation set

In [ ]:
lr_clf_grid = LogisticRegression(C=73.68423684210526)
lr_clf_grid.fit(train_features, train_labels)
lr_clf_grid.score(val_features, val_labels)

0.5383001049317944

In [ ]:
log_loss(val_labels, lr_clf_grid.predict_proba(val_features)) 
# second argument gives prob estimates for each class

1.1148371777444448

### Get accuracy and loss on test set

In [ ]:
lr_clf_grid.score(features_test, labels_test)

0.5326965466568699

In [ ]:
log_loss(labels_test, lr_clf_grid.predict_proba(features_test)) 
# second argument gives prob estimates for each class

1.0779853236179722

## Compare against dummy

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.537 (+/- 0.00)


# Clinical Diagnosis Prediction

https://huggingface.co/bvanaken/CORe-clinical-diagnosis-prediction

In [ ]:
# Load pretrained model/tokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
tokenizer = AutoTokenizer.from_pretrained("bvanaken/CORe-clinical-diagnosis-prediction")
model = AutoModelForSequenceClassification.from_pretrained("bvanaken/CORe-clinical-diagnosis-prediction")

Downloading:   0%|          | 0.00/377k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
tokenized = df['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)

padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])

np.array(padded).shape

attention_mask = np.where(padded != 0, 1, 0)
attention_mask.shape

input_ids = torch.tensor(padded)  
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
    last_hidden_states = model(input_ids, attention_mask=attention_mask)

features = last_hidden_states[0][:,:].numpy()
labels = df['label']

In [ ]:
tokenized_test = df_test['tweet'].apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))

max_len = 0
for i in tokenized_test.values:
    if len(i) > max_len:
        max_len = len(i)

padded_test = np.array([i + [0]*(max_len-len(i)) for i in tokenized_test.values])

np.array(padded_test).shape

attention_mask_test = np.where(padded_test != 0, 1, 0)
attention_mask_test.shape

input_ids_test = torch.tensor(padded_test)  
attention_mask_test = torch.tensor(attention_mask_test)

with torch.no_grad():
    last_hidden_states = model(input_ids_test, attention_mask=attention_mask_test)

features_test = last_hidden_states[0][:,:].numpy()
labels_test = df_test['label']

## Without parameter grid search

### Validation Set

In [ ]:
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size=0.3, random_state=11)

### Get accuracy and loss on validation set

In [ ]:
lr_clf = LogisticRegression()
lr_clf.fit(train_features, train_labels)
lr_clf.score(val_features, val_labels)

0.6715634837355718

In [ ]:
log_loss(val_labels, lr_clf.predict_proba(val_features)) 
# second argument gives prob estimates for each class

0.8470656461744701

### Get accuracy and loss on test set

In [ ]:
lr_clf.score(features_test, labels_test)

0.6737692872887583

In [ ]:
log_loss(labels_test, lr_clf.predict_proba(features_test)) 
# second argument gives prob estimates for each class

0.8175803741050877

## With parameter grid search

In [ ]:
parameters = {'C': np.linspace(0.0001, 100, 20)}
grid_search = GridSearchCV(LogisticRegression(), parameters)
grid_search.fit(train_features, train_labels)

print('best parameters: ', grid_search.best_params_)
print('best scores: ', grid_search.best_score_)

best parameters:  {'C': 73.68423684210526}
best scores:  0.6809130478793399


### Get accuracy and loss on validation set

In [ ]:
lr_clf_grid = LogisticRegression(C=73.68423684210526)
lr_clf_grid.fit(train_features, train_labels)
lr_clf_grid.score(val_features, val_labels)

0.6736621196222455

In [ ]:
log_loss(val_labels, lr_clf_grid.predict_proba(val_features)) 
# second argument gives prob estimates for each class

0.8623499973292076

### Get accuracy and loss on test set

In [ ]:
lr_clf_grid.score(features_test, labels_test)

0.6730345334313005

In [ ]:
log_loss(labels_test, lr_clf_grid.predict_proba(features_test)) 
# second argument gives prob estimates for each class

0.8289552240799322

## Compare against dummy

In [ ]:
from sklearn.dummy import DummyClassifier
clf = DummyClassifier()

scores = cross_val_score(clf, train_features, train_labels)
print("Dummy classifier score: %0.3f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Dummy classifier score: 0.537 (+/- 0.00)
